In [1]:
import numpy as np
import pandas as pd
from Freezer import getMyPosition as getPosition
import seaborn as sns
import matplotlib.pyplot as plt

nInst = 0
nt = 0

# Commission rate
commRate = 0.0025 # was 0.0050

# Dollar position limit (maximum absolute dollar value of any individual stock position)
dlrPosLimit = 10000

timeOut=600 

pricesFile="./prc500.txt"
prcAll = df=pd.read_csv(pricesFile, sep='\s+', header=None, index_col=None)

In [2]:
y_s = [i for i in range(250)]
prcNormalized = pd.DataFrame()
for i in range(100):
    prcNormalized[i] = prcAll[i].pct_change()
prcNormalized


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.003462,-0.004082,0.000970,-0.005402,-0.002936,-0.005186,0.000555,0.003217,-0.003626,-0.002025,...,0.003442,-0.000261,-0.000365,-0.003159,0.006254,0.000000,0.001356,0.002698,0.000900,0.006083
2,-0.001479,0.004098,0.002262,-0.002173,0.003155,0.011730,0.000924,-0.004276,0.003639,0.006492,...,-0.007622,0.000782,0.000730,-0.002465,-0.004351,0.000158,-0.000193,0.000897,-0.001199,0.011366
3,-0.001234,0.004082,-0.001289,0.002722,-0.000839,0.009662,0.000185,-0.006441,0.002039,-0.001613,...,0.000960,0.000260,0.000182,-0.001765,-0.010612,0.000475,0.001161,-0.001792,0.011708,0.000956
4,-0.005930,-0.002033,-0.001614,-0.001629,0.009022,0.005742,0.000739,0.005943,0.007463,0.000808,...,0.012085,-0.000521,-0.000547,0.007072,-0.003470,0.000000,0.001353,0.000000,0.001780,-0.014095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.002287,0.000000,-0.000368,-0.004279,0.000675,-0.005689,-0.000772,0.000782,0.001310,-0.002443,...,-0.007699,-0.000818,0.000000,-0.000378,0.018619,-0.000165,0.000608,0.001235,0.005022,0.000483
496,0.000000,0.000000,0.000736,-0.001228,-0.003821,-0.009044,0.001159,0.000000,0.001832,0.001837,...,-0.016139,0.000819,0.000579,0.005667,-0.001523,0.000165,-0.000608,0.001233,0.000937,0.003138
497,-0.005323,0.006667,-0.000735,-0.006761,-0.003159,0.000000,0.000579,0.003909,-0.001567,0.000407,...,-0.009464,0.000545,0.000000,0.001878,-0.007628,0.000165,0.001216,-0.004926,0.000312,0.014918
498,-0.002548,-0.003311,-0.000368,0.001856,-0.002490,-0.001118,-0.000771,-0.007009,-0.001832,-0.003666,...,-0.015605,0.000000,-0.000772,-0.008249,0.001537,0.000000,-0.000202,-0.002475,0.002495,0.001897


In [30]:


def giveCorrelatedChanges(prcPct, stockName):
    numStocks = 100
    backtrackDays = 30
    leaderDays = 10
    currentTotal = 0
    curStock = prcPct[stockName]
    curStock = curStock.iloc[-backtrackDays:]
    changes = []
    for i in range(numStocks):
        if i == stockName:
            changes.append((1,0, i))
            currentTotal += 1
            continue
        bestCorr = 0
        shift = None
        for j in range(1):
            curComp = prcPct[i].shift(j).iloc[-backtrackDays:]
            curCorr = curStock.corr(curComp)
            if abs(curCorr) > abs(bestCorr):
                bestCorr = curCorr 
                shift = j
        if abs(bestCorr) > 0.7:
            currentTotal += abs(bestCorr)
            changes.append((bestCorr, shift, i))
    return currentTotal, changes


In [32]:
prcPct = prcAll.iloc[:66,:]
stockName = 5
giveCorrelatedChanges(prcPct, stockName)

(30.709870016838842,
 [(-0.7101632294633161, 0, 0),
  (0.7410205857183233, 0, 1),
  (-0.9301928377259944, 0, 3),
  (1, 0, 5),
  (0.7892364075912393, 0, 7),
  (-0.8806235957085357, 0, 19),
  (0.8781352381712081, 0, 21),
  (0.9062690501281919, 0, 29),
  (-0.8668956330970435, 0, 30),
  (0.9117791561923216, 0, 31),
  (0.8286530289023635, 0, 32),
  (0.9169003485695149, 0, 33),
  (0.8982339969976818, 0, 36),
  (0.9073084827164934, 0, 41),
  (-0.8765083624942768, 0, 42),
  (0.8719495405814295, 0, 44),
  (0.9284535385977072, 0, 49),
  (-0.7241872228756313, 0, 50),
  (0.8332311314553932, 0, 52),
  (-0.9013847267923782, 0, 53),
  (0.8451948133596658, 0, 54),
  (-0.8076407270589916, 0, 55),
  (-0.8050286144015507, 0, 56),
  (0.7188223642458098, 0, 59),
  (0.7923749770782355, 0, 60),
  (0.7535336092560423, 0, 62),
  (0.778907503201952, 0, 66),
  (0.8344663455080228, 0, 69),
  (0.8738777094186018, 0, 73),
  (0.7696054967137633, 0, 74),
  (0.7356979215535547, 0, 76),
  (-0.8021551402269858, 0, 82),


In [27]:
def getMyPosition (prcSoFar):
    global currentPos, EWMAs
    curEWMA = []
    prcPct = pd.DataFrame()
    
    for i in range(len(prcSoFar)):
        close_prc = prcSoFar[i][-1]

        # days looking back for moving avg
        span = 15
        span_ceiling = 30
        span_floor = 10
        
        # changing span depending on volatility
        if len(prcSoFar[0]) > 31:

            todayVola = np.std(prcSoFar[i][-31:-2])
            yesterdayVola = np.std(prcSoFar[i][-30:])

            normVola = (todayVola - yesterdayVola) / todayVola
            span = round(span * (1 + normVola))

            if span > span_ceiling:
                span = span_ceiling
            elif span < span_floor:
                span = span_floor


        # calculate moving avg
        prc_series = pd.DataFrame(prcSoFar[i], columns=['Close'])
        prc_series['EWMA'] = prc_series['Close'].ewm(span=span).mean()
        prcPct[i] = prc_series['Close'].pct_change()

        # setting boundaries for trade criteria
        avg_prc = sum(prcSoFar[i]) / len(prcSoFar[i])
        std_multiple = np.std(prcSoFar[i])
        shift = avg_prc*std_multiple

        curEWMA.append((prc_series["EWMA"].iloc[-1],shift))
    
    EWMAs.append(curEWMA)

    # setting trade criteria
    for i in range(len(prcSoFar)):
        curStockPos = 0
        currentTotal, changes = giveCorrelatedChanges(prcPct, i)
        for change in changes:
            curCorr, leadShift, curStock = change
            curEWMA, curShift = EWMAs[-leadShift-1][curStock]
            if curEWMA + curShift  <= prcSoFar[i][-leadShift-1]:
                curStockPos -= round(8500 / close_prc)*curCorr/currentTotal
            elif curEWMA - curShift  >= prcSoFar[i][-leadShift-1]:
                curStockPos += round(8500 / close_prc)*curCorr/currentTotal
        currentPos[i] += curStockPos

    return currentPos